In [26]:
import numpy as np

# Step 1: Low-Amplitude Filtering
def low_amplitude_filtering(power_series, threshold=3.0):
    """
    Filters out signals with an amplitude lower than the threshold.
    
    Args:
    power_series (numpy array): The time series of aggregated power data.
    threshold (float): The filtering threshold to remove small power appliances.
    
    Returns:
    numpy array: The filtered power time series.
    """
    # Remove noise and filter out low-power signals
    filtered_power = np.where(power_series >= threshold, power_series, 0)
    return filtered_power

# Step 2: Short-Term PCE Filtering
def short_term_pce_filtering(power_series, min_duration=20, max_duration=3.72 * 60):
    """
    Filters out events that are too short or too long to be EV charging events.
    
    Args:
    power_series (numpy array): The filtered power time series from step 1.
    min_duration (int): Minimum duration in minutes for an event to be considered EV charging.
    max_duration (int): Maximum duration in minutes for an event to be considered EV charging.
    
    Returns:
    numpy array: The filtered power time series after removing short-duration events.
    """
    filtered_power = np.copy(power_series)
    current_event_start = None
    
    # Loop over the power series to identify events
    for i in range(len(power_series)):
        if power_series[i] > 0:
            if current_event_start is None:
                current_event_start = i
        else:
            if current_event_start is not None:
                # Calculate the event duration
                duration = i - current_event_start
                if duration < min_duration or duration > max_duration:
                    # Filter out the event if duration does not match EV charging event duration
                    filtered_power[current_event_start:i] = 0
                current_event_start = None

    return filtered_power

# Step 3: Event Detection and Matching
def detect_and_match_events(power_series, edge_threshold=1.5, skip_points=3):
    """
    Detects and matches events by identifying the edges in the power series using skipping difference.
    
    Args:
    power_series (numpy array): The filtered power time series from previous steps.
    edge_threshold (float): The threshold for detecting a significant event edge.
    skip_points (int): The number of points to skip in the skipping difference method.
    
    Returns:
    list of tuples: A list of detected events in the format (start_index, end_index, event_magnitude).
    """
    events = []
    event_start = None
    
    for i in range(len(power_series) - skip_points):
        # Calculate the skipping difference between power points
        power_diff = power_series[i + skip_points] - power_series[i]
        
        # Detect rising edge
        if power_diff >= edge_threshold:
            if event_start is None:  # Start of a new event
                event_start = i
        
        # Detect falling edge and match it with the rising edge
        elif power_diff <= -edge_threshold:
            if event_start is not None:  # We have a complete event
                event_end = i + skip_points
                event_magnitude = power_series[event_end] - power_series[event_start]
                # Check if the rising and falling edges are part of the same event
                if abs(event_magnitude) > edge_threshold:
                    events.append((event_start, event_end, event_magnitude))
                event_start = None  # Reset for the next event
    
    return events

# Example Usage:
# Example time series of aggregated power data (in kW)
power_series = np.array([0, 0.5, 1, 2, 4, 6, 0, 0, 3, 5, 7, 0, 0.8, 2, 0])

# Step 1: Low-Amplitude Filtering
filtered_power_step1 = low_amplitude_filtering(power_series, threshold=2.5)

# Step 2: Short-Term PCE Filtering
filtered_power_step2 = short_term_pce_filtering(filtered_power_step1, min_duration=20, max_duration=3.72 * 60)

# Step 3: Event Detection and Matching
events = detect_and_match_events(filtered_power_step2, edge_threshold=1.5, skip_points=3)

# Display the detected events
print("Detected Events (start_index, end_index, magnitude):")
for event in events:
    print(event)


Detected Events (start_index, end_index, magnitude):


In [25]:
filtered_power_step2

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])